In [1]:
#IMPORTAMOS LAS LIBRERIAS BASADAS EN TENSORFLOW Y KERAS
import sys
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K

#LIMPIAMOS SESION DE KERAS PARA EMPEZAR DESDE 0 EN MEMORIA
K.clear_session()

#GENERAMOS 2 VARIABLES CON CADA UNA DE LAS CARPETAS TANTO DE ENTRENAMIENTO COMO DE TEST CON FOTOGRAFIAS
data_entrenamiento = './Dataset/train'
data_validacion = './Dataset/test'

#DECLARACION DE VARIABLES CON VALORES QUE PARAMETRIZARAN EL DESEMPEÑO DEL ENTRENAMIENTO DEL MODELO DE CLASIFICACION DE FOTOS
epocas=150 #NUMERO DE EPOCHS O ITERACCIONES SOBRE TODO EL SET DE DATOS PARA EL TRAINING
longitud, altura = 100,100 #REDIMENSIONAMIENTO DE FOTOGRAFIAS
batch_size = 16 # NUMERO DE FOTOGRAFIAS A GESTIONAR EN CADA UNO DE LOS PASOS DE PROCESO
pasos = 200 #NUMERO DE STEPS PARA PROCESAR LA INFORMACION EN CADA UNA DE LAS EPOCAS
validation_steps = 350 #NUMERO DE STEPS PARA VALIDACION CON LOS DATOS DE TEST
filtrosConv1 = 32 #AL HACER UNA CONVOLACION LA IMAGEN TENDRA PROFUNDIDAD DE 32
filtrosConv2 = 64 #AL HACER OTRA CONVOLACION LA IMAGEN TENDRA PROFUNDIDAD DE 64
tamano_filtro1 = (3, 3) #TAMAÑO DE FILTROS EN LA PRIMERA CONVULACION
tamano_filtro2 = (2, 2) #TAMAÑO DE FILTROS EN LA SEGUNDACONVULACION
tamano_pool = (2, 2) #TAMAÑO DEL FILTRO A USAR EN EL MAXPOOLING
clases = 14 #CANTIDAD DE CLASES DE FOTOGRAFIAS A PROCESAR
lr = 0.0001 #LEARNING RATE PARA EL PROCESO DE ENTRENAMIENTO EN AJUSTES A HACER PARA ACERCARSE A UNA SOLUCION OPTIMA

In [2]:
#FUNCION IMAGEDATAGENERATOR QUE PROPORCIONARA MODIFICACION A LAS FOTOGRAFIAS PARA OPTIMIZAR EL PROCESO DE ENTRENAMIENTO
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)
#FUNCION PARA CONFIGURACION DEL PROCESO DE RECUPERACION DE INFORMACION DE FOTOS PARA ENTRENAMIENTO
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')
#FUNCION PARA CONFIGURACION DEL PROCESO DE RECUPERACION DE INFORMACION DE FOTOS PARA TEST
validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 117 images belonging to 4 classes.
Found 1 images belonging to 4 classes.


In [3]:
#FUNCION PARA EMPEZAR EL MODELO Y SU CONSTRUCCION
cnn = Sequential()
#PRIMERA CAPA
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
#SEGUNDA CAPA
cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
#RECONFIGURACION DE LA INFORMACION EN DATOS RECONOCIBLES POR EL MODELO
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))
#COMPILACION DE LA INFORMACION DENTRO DEL MODELO PARA SU ENTRENAMIENTO 
cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(learning_rate=lr),
            metrics=['accuracy'])

In [ ]:
#ENTRENAMIENTO DEL MODELO SEGUN LOS DATOS INGRESADOS COMO PARAMETROS
cnn.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

In [5]:
#CREACION Y ALMACENAMIENTO DEL RESULTANTE DEL ENTRENAMIENTO PARA RECONOCIMIENTO DE FOTOGRAFIAS
target_dir = './'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
